In [1]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from commit_transformer.tokenizer import build_vocab
from commit_transformer.preprosser import CommitDataset
from commit_transformer.model import CombinedModel

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'./datasets/train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'./datasets/test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
val = pd.read_csv(r'./datasets/val.csv', encoding='utf_8_sig')
val = test.replace({"label": label2id})

test.fillna('', inplace=True)
val.fillna('', inplace=True)
train.fillna('', inplace=True)

In [ ]:
# df = df[['message','diff','label']] 
# df

In [9]:
train_data = []
for index, row in train.iterrows():
  train_data.append([row['message'],row['diff'],row['label']])

val_data = []
for index, row in val.iterrows():
  val_data.append([row['message'],row['diff'],row['label']])
test_data = []
for index, row in test.iterrows():
  test_data.append([row['message'],row['diff'],row['label']])

In [10]:

# # Split the dataset
# train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# # Build vocabulary
vocab = build_vocab(train_data)

# Create Datasets and DataLoaders
train_dataset = CommitDataset(train_data, vocab)
val_dataset = CommitDataset(val_data, vocab)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:


# Initialize the model
vocab_size = len(vocab)
embed_dim = 128  # Dimension for embeddings
num_heads = 8
hidden_dim = 512
num_layers = 6
dropout = 0.3

model = CombinedModel(vocab_size, embed_dim, num_heads, hidden_dim, num_layers, dropout)

# Train the model
model.trainer(train_loader, val_loader, num_epochs=10)

In [11]:
model.evaluate(val_loader)

Validation Accuracy: 0.7066578773895847
Precision: 0.5896805896805897
Recall: 0.8121827411167513
F1-Score: 0.6832740213523132
